In [1]:
import pandas as pd


In [2]:
path_global = 'http://berkeleyearth.lbl.gov/auto/Global/Land_and_Ocean_summary.txt'
temp_data_global = pd.read_csv(path_global, header=None, delimiter=' ', comment = '%')

ParserError: Error tokenizing data. C error: Expected 2 fields in line 59, saw 91


In [10]:
temp_data_global

,0
0,1850 -0.443 0.155 N...
1,1851 -0.328 0.181 N...
2,1852 -0.301 0.148 -0.3...
3,1853 -0.299 0.167 -0.3...
4,1854 -0.297 0.146 -0.3...
...,...
167,2017 0.965 0.025 0.9...
168,2018 0.892 0.028 1.0...
169,2019 1.031 0.031 0.9...
170,2020 1.058 0.031 N...
